# Attacks on Neural Networks in a Lightweight Speech Pseudonymization Pipeline

## Imports

In [ ]:
import ASR_2024_anonymization_module_learning.speaker_anonymization as pipeline
import ASR_2024_anonymization_module_learning.speaker_anonymization.optimize as pipeline_optimize

import util
from backdoored_dataset import BackdooredVCTK, CachedVCTK
from attacks.jingleback import JingleBack
from attacks.fgsm import FGSM
from attacks.pgd import PGD
from metrics import attack_success_rate, clean_accuracy_drop

import os
import gc
import warnings

import pandas as pd
import seaborn as sns

import torch
import torchaudio
from torch.utils.data import DataLoader

## Preparation

In [ ]:
pipeline_config_train = pipeline.config.Config(
    num_trials=5,
    n_speakers=10,
    n_samples_per_speaker=10,
    gender=None,
    min_age=None,
    max_age=None,
    accent=None,
    region=None
)

pipeline_config_test = pipeline.config.Config(
    num_trials=1,
    n_speakers=10,
    n_samples_per_speaker=100,
    gender=None,
    min_age=None,
    max_age=None,
    accent=None,
    region=None
)

os.makedirs(os.path.join(pipeline_config_train.BACKDOORED_FOLDER, "train"), exist_ok=True)
os.makedirs(os.path.join(pipeline_config_train.BACKDOORED_FOLDER, "test"), exist_ok=True)
os.makedirs(os.path.join(pipeline_config_train.PERTURBED_FOLDER, "fgsm"), exist_ok=True)
os.makedirs(os.path.join(pipeline_config_train.PERTURBED_FOLDER, "pgd"), exist_ok=True)

warnings.filterwarnings("ignore")

util.set_global_seed(3131)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

if str(device) == "cuda":
    print(torch.cuda.get_device_name())

## Creating Clean and Backdoored Test Datasets

In [ ]:
clean_test_set = CachedVCTK(pipeline_config_test)


TARGET_SPEAKER_ID = 0
jingleback_attack = JingleBack(source_label=None, target_label=TARGET_SPEAKER_ID)

backdoored_test_set = BackdooredVCTK(jingleback_attack, poisoning_rate=1.0, train=False, pipeline_config=pipeline_config_test)
backdoored_test_loader = DataLoader(backdoored_test_set, batch_size=1, shuffle=False)

## Assessing the Clean Models

In [ ]:
clean_asr_processor, clean_asr_model, clean_asv_model, clean_wer, clean_asv_acc, _ = pipeline_optimize.optimize_audio_effects(pipeline_config_train, stop_after_model_evaluation=True)
clean_asv_asr, _ = attack_success_rate(clean_asv_model, backdoored_test_loader, target_label=TARGET_SPEAKER_ID, device=device)

print("WER:", clean_wer)
print("ASV Acc:", clean_asv_acc)
print("ASV ASR:", clean_asv_asr)

## Assessing Backdoored Models

In [ ]:
poisoning_rates = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]

asr_word_error_rates = []
asr_word_error_rate_increases = []
asv_accuracies = []
asv_attack_success_rates = []
asv_clean_accuracy_drops = []

for poisoning_rate in poisoning_rates:
    backdoored_train_set = BackdooredVCTK(jingleback_attack, poisoning_rate=poisoning_rate, train=True, pipeline_config=pipeline_config_train)
    backdoored_train_loader = DataLoader(backdoored_train_set, batch_size=1, shuffle=True)
    
    asr_processor, asr_model, asv_model, mean_wer, mean_asv_acc, _ = pipeline_optimize.optimize_audio_effects(pipeline_config_train, backdoored_vctk=backdoored_train_set, stop_after_model_evaluation=True)
    
    weri = mean_wer - clean_wer
    asr, _ = attack_success_rate(asv_model, backdoored_test_loader, target_label=TARGET_SPEAKER_ID, device=device)
    cad = clean_accuracy_drop(clean_asv_model, asv_model, clean_test_set)
    
    asr_word_error_rates.append(mean_wer)
    asr_word_error_rate_increases.append(weri)
    asv_accuracies.append(mean_asv_acc)
    asv_attack_success_rates.append(asr)
    asv_clean_accuracy_drops.append(cad)
    
    print("Poisoning Rate:", poisoning_rate)
    print("    ASR Word Error Rate:", mean_wer)
    print("    ASR Word Error Rate Increase:", weri)
    print("    ASV Accuracy:", mean_asv_acc)
    print("    ASV Attack Success Rate:", asr)
    print("    ASV Clean Accuracy Drop:", cad)

## Visualizing the Backdoor

In [ ]:
visualizations = []
visualizations.append([0, "WER", clean_wer])
visualizations.append([0, "Acc", clean_asv_acc])
visualizations.append([0, "ASR", clean_asv_asr])

for i in range(len(poisoning_rates)):
    visualizations.append([poisoning_rates[i], "WER", asr_word_error_rates[i]])
    visualizations.append([poisoning_rates[i], "WERI", asr_word_error_rate_increases[i]])
    visualizations.append([poisoning_rates[i], "Acc", asv_accuracies[i]])
    visualizations.append([poisoning_rates[i], "ASR", asv_attack_success_rates[i]])
    visualizations.append([poisoning_rates[i], "CAD", asv_clean_accuracy_drops[i]])
    
visualizations_df = pd.DataFrame(visualizations, columns=["Poisoning Rate", "Metric", "Value"])
display(visualizations_df)

sns.set_theme(rc={'figure.figsize':(10, 5)})
sns.pointplot(data=visualizations_df, x="Poisoning Rate", y="Value", hue="Metric", hue_order=["WER", "WERI", "Acc", "CAD", "ASR"])

## Assessing Evasion Attacks

In [ ]:
clean_utterances, speaker_ids, transcriptions, _, _ = clean_test_set[:]

gc.collect()
torch.cuda.empty_cache()

### FGSM

In [ ]:
fgsm = FGSM(clean_asv_model, eps=8/255)
fgsm.set_mode_targeted_by_label(quiet=True)

fgsm_utterances = []
for index, utterance in enumerate(clean_utterances):
    fgsm_utterance = fgsm(utterance, TARGET_SPEAKER_ID)
    fgsm_utterances.append(fgsm_utterance)
    
    utterance_file_name = f"index{index}_speaker{speaker_ids[index]}_fgsm_target{TARGET_SPEAKER_ID}.wav"
    utterance_file_path = "d:/Datasets/vctk/perturbed_audio/fgsm/" + utterance_file_name
    torchaudio.save(uri=utterance_file_path, src=fgsm_utterance.cpu(), sample_rate=16000)

predicted_transcriptions, predicted_speakers, fgsm_wer, fgsm_asv_acc, _ = pipeline_optimize.evaluate_asr_and_asv(
    audio_data=[(fgsm_utterance.squeeze().to(device), None) for fgsm_utterance in fgsm_utterances],
    transcriptions=transcriptions,
    speakers=speaker_ids,
    asr_processor=clean_asr_processor,
    asr_model=clean_asr_model,
    asv_model=clean_asv_model,
    CONFIG=pipeline_config_test
)

### PGD

In [ ]:
pgd = PGD(clean_asv_model, eps=8/255, alpha=2/255, steps=10, random_start=True)
pgd.set_mode_targeted_by_label(quiet=True)

pgd_utterances = []
for index, utterance in enumerate(clean_utterances):
    pgd_utterance = pgd(utterance, TARGET_SPEAKER_ID)
    pgd_utterances.append(pgd_utterance)
    
    utterance_file_name = f"index{index}_speaker{speaker_ids[index]}_pgd_target{TARGET_SPEAKER_ID}.wav"
    utterance_file_path = "d:/Datasets/vctk/perturbed_audio/pgd/" + utterance_file_name
    torchaudio.save(uri=utterance_file_path, src=pgd_utterance.cpu(), sample_rate=16000)

predicted_transcriptions, predicted_speakers, pgd_wer, pgd_asv_acc, _ = pipeline_optimize.evaluate_asr_and_asv(
    audio_data=[(pgd_utterance.squeeze().to(device), None) for pgd_utterance in pgd_utterances],
    transcriptions=transcriptions,
    speakers=speaker_ids,
    asr_processor=clean_asr_processor,
    asr_model=clean_asr_model,
    asv_model=clean_asv_model,
    CONFIG=pipeline_config_test
)